In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import pydataset
from pydataset import data

import warnings
warnings.filterwarnings('ignore')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 11:05:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## EXERCISE
### Create a spark data frame that contains your favorite programming languages.

### a)The name of the column should be language

In [2]:
# Creating pd dataframe with my favorite programming languages
df = pd.DataFrame ({ 'language': ['Python', 'SQL','JavaScript', 'Java' 'HTML', 'R']})
df

,language
0,Python
1,SQL
2,JavaScript
3,JavaHTML
4,R


In [3]:
# Creating spark dataframe with my favorite programming languages
spark_df = spark.createDataFrame(df)
spark_df.show()            

+----------+
|  language|
+----------+
|    Python|
|       SQL|
|JavaScript|
|  JavaHTML|
|         R|
+----------+



### b) View the schema of the dataframe

In [4]:
# Output schema of dataframe
spark_df.printSchema()

root
 |-- language: string (nullable = true)



### c) Output the shape of the dataframe

In [5]:
#checking the shape of the spark dataframe
print((spark_df.count(), len(spark_df.columns)))

(5, 1)


### d) Show the first 5 records in the dataframe

In [6]:
# check spark df
spark_df.show(5)

+----------+
|  language|
+----------+
|    Python|
|       SQL|
|JavaScript|
|  JavaHTML|
|         R|
+----------+



### 2. Load the mpg dataset as a spark dataframe.

### a) Create 1 column of output that contains a message like the one below:
 - The 1999 audi a4 has a 4 cylinder engine.
 - For each vehicle

In [7]:
from pyspark.sql.functions import col, expr, concat, lit


In [8]:
#define mpg df
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
#function to create column output
spark_df = mpg.select(concat(lit('The '), 
                       mpg.year,
                       lit(' '),
                        mpg.manufacturer,
                       lit(' '), 
                       mpg.model, 
                       lit(' has a '),
                       mpg.cyl, 
                       lit(' cylinder engine.')).alias("column"))
spark_df.show(truncate=False)

+--------------------------------------------------------------+
|column                                                        |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

### b) Transform the trans column so that it only contains either manual or auto.

In [10]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [11]:

mpg.select(
    "trans",
    regexp_extract('trans',r'(\w{4,6})',1)
).show(5)

+----------+-----------------------------------+
|     trans|regexp_extract(trans, (\w{4,6}), 1)|
+----------+-----------------------------------+
|  auto(l5)|                               auto|
|manual(m5)|                             manual|
|manual(m6)|                             manual|
|  auto(av)|                               auto|
|  auto(l5)|                               auto|
+----------+-----------------------------------+
only showing top 5 rows



### 3. Load the tips dataset as a spark dataframe.

In [12]:
#load tips dataframe showing first 5 rows
spark_df = spark.createDataFrame(pydataset.data('tips'))
spark_df.show(5)


+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### a) What percentage of observations are smokers?

In [13]:
# Calculate percentage of observations that are smokers.
spark_df.where(spark_df.smoker == 'Yes').count() / spark_df.count()

0.38114754098360654

### b) Create a column that contains the tip percentage

In [14]:
# Let's create a tip percentage column.
spark_df = spark_df.withColumn('tip_percentage', spark_df.tip / spark_df.total_bill)
spark_df.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



### c) Calculate the average tip percentage for each combination of sex and smoker.

In [15]:
# Calculate average tip percentage for sex and smoker groups.
spark_df.groupBy('sex', 'smoker').agg(F.mean(spark_df.tip_percentage)).show()

+------+------+-------------------+
|   sex|smoker|avg(tip_percentage)|
+------+------+-------------------+
|  Male|    No| 0.1606687151291298|
|Female|    No| 0.1569209707691836|
|  Male|   Yes|0.15277117520248512|
|Female|   Yes|0.18215035269941032|
+------+------+-------------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [16]:
from vega_datasets import data


In [17]:
#create seattle weather dataframe
spark_df = spark.createDataFrame(data.seattle_weather())
spark_df.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



### a) Convert the temperatures to fahrenheit.

In [18]:
# Convert temperatures to fahrenheit.
spark_df = spark_df.withColumn('temp_max', spark_df.temp_max * 1.8 + 32).withColumn('temp_min', spark_df.temp_min * 1.8 + 32)
spark_df.show(5)

+-------------------+-------------+------------------+--------+----+-------+
|               date|precipitation|          temp_max|temp_min|wind|weather|
+-------------------+-------------+------------------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|55.040000000000006|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|             51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|             53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|             53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|48.019999999999996|   37.04| 6.1|   rain|
+-------------------+-------------+------------------+--------+----+-------+
only showing top 5 rows



### b) Which month has the most rain, on average?

In [19]:
# Calculating the month with most rain on average.
spark_df.withColumn('month', F.month('date'))\
    .withColumn('year', F.year('date'))\
    .where(spark_df.weather == 'rain')\
    .groupBy('month', 'year')\
    .agg(F.sum('precipitation').alias('total_rain'))\
    .groupBy('month')\
    .agg(F.mean('total_rain').alias('avg_total_rain'))\
    .sort('avg_total_rain', ascending = False).first()

Row(month=11, avg_total_rain=210.5)

### c) Which year was the windiest?

In [20]:
# Calculating windiest year.
spark_df.withColumn('year', F.year('date'))\
    .groupBy('year')\
    .agg(F.sum('wind').alias('total_winds'))\
    .sort('total_winds', ascending = False).show(1)

+----+-----------+
|year|total_winds|
+----+-----------+
|2012|     1244.7|
+----+-----------+
only showing top 1 row



### d) What is the most frequent type of weather in January?

In [21]:
# Calculating most frequent type of weather in January.
spark_df.where(F.month('date') == 1)\
    .groupBy('weather')\
    .count()\
    .sort('count', ascending = False).show(1)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



### e) What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [22]:
# calculating the average high and low temperature on sunny days in July in 2013 and 2014
spark_df.where((spark_df.weather == 'sun')\
         & (F.month('date') == 7)\
         & ((F.year('date') == 2013)\
         | (F.year('date') == 2014)))\
    .select(F.mean('temp_max'), F.mean('temp_min')).show()

+-----------------+-----------------+
|    avg(temp_max)|    avg(temp_min)|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



### f) What percentage of days were rainy in q3 of 2015?

In [23]:
# calculating percentage of days that were rainy in q3 of 2015
from pyspark.sql.window import Window

spark_df.where((F.year('date') == 2015) & (F.quarter('date') == 3))\
    .groupBy('weather')\
    .count()\
    .withColumn('percentage', F.col('count') / F.sum('count').over(Window.partitionBy())).show()

22/10/21 11:05:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 11:05:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 11:05:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 11:05:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 11:05:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 11:05:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/21 1

### g) For each year, find what percentage of days it rained (had non-zero precipitation).

In [24]:
# calculating percentage of days it rained (had non-zero precipitation).

spark_df.withColumn('year', F.year('date'))\
    .withColumn('non_zero_precipitation', (spark_df.precipitation > 0).cast('int'))\
    .groupBy('year')\
    .agg(F.mean('non_zero_precipitation')).show()

+----+---------------------------+
|year|avg(non_zero_precipitation)|
+----+---------------------------+
|2012|        0.48360655737704916|
|2013|        0.41643835616438357|
|2014|          0.410958904109589|
|2015|        0.39452054794520547|
+----+---------------------------+

